In [5]:
import pandas as pd
import numpy as np

def analisar_variavel_completa(csv_path, dicionario_path):
    """
    Analisa cada variável do CSV cruzando com o dicionário de dados
    """
    
    # Carregar dados
    print("📊 Carregando dados...")
    df_dados = pd.read_csv(csv_path)
    df_dicionario = pd.read_excel(dicionario_path)
    
    print(f"✅ CSV carregado: {df_dados.shape[0]} linhas x {df_dados.shape[1]} colunas")
    print(f"✅ Dicionário carregado: {df_dicionario.shape[0]} variáveis")
    
    # Verificar colunas do dicionário
    print("\n📋 Colunas do dicionário:")
    print(df_dicionario.columns.tolist())
    
    # Função para analisar uma variável específica
    def analisar_variavel(nome_variavel):
        print(f"\n{'='*80}")
        print(f"🔍 ANÁLISE: {nome_variavel}")
        print(f"{'='*80}")
        
        # Buscar descrição no dicionário
        descricao_row = df_dicionario[df_dicionario['variavel'] == nome_variavel]
        
        if not descricao_row.empty:
            descricao = descricao_row['descricao'].iloc[0]
            print(f"📝 DESCRIÇÃO: {descricao}")
        else:
            print(f"⚠️  VARIÁVEL NÃO ENCONTRADA NO DICIONÁRIO")
        
        # Verificar se variável existe no dataset
        if nome_variavel not in df_dados.columns:
            print(f"❌ VARIÁVEL NÃO EXISTE NO DATASET")
            return
        
        # Análise da variável no dataset
        coluna = df_dados[nome_variavel]
        
        # Informações básicas
        total_registros = len(coluna)
        valores_ausentes = coluna.isnull().sum()
        valores_validos = total_registros - valores_ausentes
        pct_validos = (valores_validos / total_registros) * 100
        pct_ausentes = (valores_ausentes / total_registros) * 100
        
        print(f"\n📊 INFORMAÇÕES BÁSICAS:")
        print(f"   • Tipo: {coluna.dtype}")
        print(f"   • Valores válidos: {valores_validos} ({pct_validos:.1f}%)")
        print(f"   • Valores ausentes: {valores_ausentes} ({pct_ausentes:.1f}%)")
        print(f"   • Valores únicos: {coluna.nunique()}")
        
        # Mostrar todos os valores únicos da variável
        if valores_validos > 0:  # Só se há dados válidos
            valores_unicos = coluna.dropna().value_counts()
            
            # Limitar a 10 se houver muitos valores
            if len(valores_unicos) > 10:
                valores_para_mostrar = valores_unicos.head(10)
                mostrar_mais = True
            else:
                valores_para_mostrar = valores_unicos
                mostrar_mais = False
            
            print(f"\n🏷️ VALORES: ", end="")
            
            # Criar lista dos valores
            lista_valores = []
            for valor in valores_para_mostrar.index:
                count = valores_para_mostrar[valor]
                pct = (count / valores_validos) * 100
                lista_valores.append(f"{valor} ({count}, {pct:.1f}%)")
            
            # Mostrar valores separados por vírgula
            print(", ".join(lista_valores))
            
            # Indicar se há mais valores
            if mostrar_mais:
                restantes = len(valores_unicos) - 10
                print(f"   ... e mais {restantes} valores únicos")
        
        else:
            print(f"\n⚠️ NENHUM VALOR VÁLIDO")
        
        # Recomendação de ação
        print(f"\n💡 RECOMENDAÇÃO PRELIMINAR:")
        
        # Verificar se é tautológica (baseado no nome)
        tautologicas = ['k12_tempo', 'k28_aleitamento', 'k29_alimentacao', 
                       'k22_amamentou', 'k245_utilizou_mamadeira']
        
        if nome_variavel in tautologicas:
            print(f"   🚨 REMOVER - Variável tautológica identificada")
        elif nome_variavel == 'id_anon':
            print(f"   🚨 REMOVER - Variável de identificação")
        elif coluna.isnull().sum()/len(coluna) > 0.3:
            print(f"   ⚠️  AVALIAR - Mais de 30% de valores ausentes")
        elif nome_variavel == 'k25_mamadeira':
            print(f"   🔄 TRANSFORMAR - Recodificar para exposição à mamadeira")
        elif coluna.dtype == 'object' and coluna.nunique() > 15:
            print(f"   🔄 TRANSFORMAR - Alta cardinalidade, considerar target encoding")
        elif coluna.dtype == 'object':
            print(f"   🔄 TRANSFORMAR - Aplicar one-hot encoding")
        else:
            print(f"   ✅ MANTER - Variável adequada para ML")
    
    return analisar_variavel

# Criar função de análise
analisar_variavel = analisar_variavel_completa(
    '/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/7-limpeza do banco de dados/cs.csv',
    '/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/4-Dicionario-ENANI-2019 (1).xlsx'
)

# Exemplo de uso para analisar uma variável específica:
# analisar_variavel('aleitamento_materno_exclusivo')

# Para analisar todas as variáveis em sequência:
def analisar_todas_variaveis():
    df_dados = pd.read_csv('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/7-limpeza do banco de dados/cs.csv')
    
    print("🔍 INICIANDO ANÁLISE DE TODAS AS VARIÁVEIS")
    print(f"Total de variáveis: {len(df_dados.columns)}")
    
    for i, variavel in enumerate(df_dados.columns, 1):
        print(f"\n🔢 [{i}/{len(df_dados.columns)}]")
        analisar_variavel(variavel)
        
        # Pausa para leitura (opcional)
        if i % 5 == 0:  # A cada 5 variáveis
            resposta = input(f"\n⏸️  Analisadas {i} variáveis. Continuar? (s/n): ")
            if resposta.lower() != 's':
                break

# Para criar um resumo de todas as variáveis do CSV com suas descrições:
def verificar_variaveis_no_dicionario():
    """
    Verifica quais variáveis do CSV existem no dicionário e traz suas descrições
    """
    df_dados = pd.read_csv('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/7-limpeza do banco de dados/cs.csv')
    df_dicionario = pd.read_excel('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/4-Dicionario-ENANI-2019 (1).xlsx')
    
    print("🔍 VERIFICANDO VARIÁVEIS DO CSV NO DICIONÁRIO")
    print(f"📊 CSV: {len(df_dados.columns)} variáveis")
    print(f"📖 Dicionário: {len(df_dicionario)} entradas")
    print("="*100)
    
    encontradas = 0
    nao_encontradas = 0
    
    for i, variavel in enumerate(df_dados.columns, 1):
        print(f"\n[{i:3d}/{len(df_dados.columns)}] {variavel}")
        
        # Buscar no dicionário
        descricao_row = df_dicionario[df_dicionario['variavel'] == variavel]
        
        if not descricao_row.empty:
            encontradas += 1
            descricao = descricao_row['descricao'].iloc[0]
            print(f"    ✅ DESCRIÇÃO: {descricao}")
            
        else:
            nao_encontradas += 1
            print(f"    ❌ NÃO ENCONTRADA NO DICIONÁRIO")
            
        # ANÁLISE DOS DADOS REAIS DO CSV
        coluna = df_dados[variavel]
        
        # Informações básicas
        total_registros = len(coluna)
        valores_ausentes = coluna.isnull().sum()
        valores_validos = total_registros - valores_ausentes
        pct_validos = (valores_validos / total_registros) * 100
        pct_ausentes = (valores_ausentes / total_registros) * 100
        
        print(f"    📊 Válidos: {valores_validos} ({pct_validos:.1f}%), Ausentes: {valores_ausentes} ({pct_ausentes:.1f}%)")
        
        # Mostrar valores únicos do CSV (limitado a 10)
        if valores_validos > 0:
            valores_unicos = coluna.dropna().value_counts()
            
            # Limitar a 10 se houver muitos valores
            if len(valores_unicos) > 10:
                valores_para_mostrar = valores_unicos.head(10)
                mostrar_mais = True
            else:
                valores_para_mostrar = valores_unicos
                mostrar_mais = False
            
            # Criar lista dos valores
            lista_valores = []
            for valor in valores_para_mostrar.index:
                count = valores_para_mostrar[valor]
                pct = (count / valores_validos) * 100
                lista_valores.append(f"{valor} ({count}, {pct:.1f}%)")
            
            # Mostrar valores
            print(f"    🏷️  VALORES: {', '.join(lista_valores)}")
            
            # Indicar se há mais valores
            if mostrar_mais:
                restantes = len(valores_unicos) - 10
                print(f"    ... e mais {restantes} valores únicos")
        
        else:
            print(f"    ⚠️  NENHUM VALOR VÁLIDO")
    
    print("\n" + "="*100)
    print(f"📊 RESUMO:")
    print(f"   ✅ Encontradas: {encontradas} variáveis")
    print(f"   ❌ Não encontradas: {nao_encontradas} variáveis")
    print(f"   📈 Taxa de correspondência: {encontradas/len(df_dados.columns)*100:.1f}%")

# Executar verificação completa
verificar_variaveis_no_dicionario()

📊 Carregando dados...


/opt/anaconda3/envs/pycaret_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


✅ CSV carregado: 1960 linhas x 173 colunas
✅ Dicionário carregado: 2165 variáveis

📋 Colunas do dicionário:
['bloco', 'variavel', 'descricao', 'origem', 'tamanho', 'tipo', 'n_decimais', 'valor', 'descricao_labels', 'imputada', 'metodo_imputacao', 'observacoes (referencias, calculos, notas)']
🔍 VERIFICANDO VARIÁVEIS DO CSV NO DICIONÁRIO
📊 CSV: 173 variáveis
📖 Dicionário: 2165 entradas

[  1/173] aleitamento_materno_exclusivo
    ❌ NÃO ENCONTRADA NO DICIONÁRIO
    📊 Válidos: 1960 (100.0%), Ausentes: 0 (0.0%)
    🏷️  VALORES: 0 (1040, 53.1%), 1 (920, 46.9%)

[  2/173] id_anon
    ✅ DESCRIÇÃO: id_anon - Código de identificação da criança
    📊 Válidos: 1960 (100.0%), Ausentes: 0 (0.0%)
    🏷️  VALORES: 10951009303 (1, 0.1%), 11231013107 (1, 0.1%), 10121030603 (1, 0.1%), 10121003303 (1, 0.1%), 10683029204 (1, 0.1%), 10703052203 (1, 0.1%), 10703023904 (1, 0.1%), 10701057704 (1, 0.1%), 10701013804 (1, 0.1%), 10138031703 (1, 0.1%)
    ... e mais 1950 valores únicos

[  3/173] a00_regiao
    ✅ 

/opt/anaconda3/envs/pycaret_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
